### OPEN API 를 이용한 챗봇 시스템 

In [3]:
!pip show openapi
# !pip install openai

Name: openapi
Version: 2.0.0
Summary: Python OpenAPI 2.0 (Swagger) object model
Home-page: https://github.com/globality-corp/openapi
Author: Globality Engineering
Author-email: engineering@globality.com
License: 
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: inflection, jsonschema
Required-by: 


---
### OPENAI API 호출 예제 

In [6]:
from openai import OpenAI
from pprint import pprint
import os
api_key=os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

client = OpenAI(api_key=api_key)
model = 'gpt-3.5-turbo-1106'

messages = [
    {'role':'system','content': 'You are a helphful assistant.'},
    {'role':'user','content': 'Who won the world series in2020?'}
]
response = client.chat.completions.create(model=model, messages=messages).model_dump()
pprint(response)

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': 'The Los Angeles Dodgers won the World '
                                     'Series in 2020, defeating the Tampa Bay '
                                     'Rays.',
                          'function_call': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1719896503,
 'id': 'chatcmpl-9gPw7bfwR2ziZFIXjy7v22TgeFRBk',
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_44132a4de3',
 'usage': {'completion_tokens': 19, 'prompt_tokens': 28, 'total_tokens': 47}}


> gpt 는 json type 으로 답을 넘겨준다. 

---
### 

 * [model dump ]
    : 메서드 체이닝 -> 메서드를 연쇄적으로 호출하는 방식 